# Lab 2: Generative Models (Generative adversarial networks)
```
- [S25] Advanced Machine Learning, Innopolis University
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
1. Conditional Generative adversarial networks
2. Bidirectional Generative Adversarial Network
3. Task 2
```

<hr>

## 1. Conditional Generative adversarial network


The Conditional Generative Adversarial Network (cGAN) is a model used in deep learning, a derivative of machine learning. It enables more precise generation and discrimination of data. The achitecture is similar to the one of vanilla GAN.


The condition for data generation is a link to the label of the data.

![Conditional GAN](https://www.researchgate.net/profile/Gerasimos-Spanakis/publication/330474693/figure/fig1/AS:956606955139072@1605084279074/GAN-conditional-GAN-CGAN-and-auxiliary-classifier-GAN-ACGAN-architectures-where-x_Q320.jpg)

### Imports

`!pip install tensorboardX`

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder, MNIST
from torchvision import transforms
from torch import autograd
from torchvision.utils import make_grid
import torchvision
from torch.utils.data import DataLoader

from tensorboardX import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 1.1 Dataset preparation

In this task we will use MNIST dataset

In [2]:
batch_size = 32
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
])



def load_dataset(batch_size = 128, root = '~/datasets/', transform=transforms.ToTensor()):
    train_dataset = torchvision.datasets.MNIST(root = root + '/MNIST', train=True,
                                               transform=transform, download=True)

    test_dataset = torchvision.datasets.MNIST(root = root + '/MNIST', train=False,
                                              transform=transform)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

train_loader, _ = load_dataset(batch_size=batch_size, transform=transform)

### 1.2 Define Discriminator model

In [21]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()

    self.label_emb = nn.Embedding(10, 10)

    self.model = nn.Sequential(
        nn.Linear(794, 1024),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

  def forward(self, x, labels):
    x = x.view(x.size(0), 784)
    c = self.label_emb(labels)
    x = torch.cat([x, c], 1)
    out = self.model(x)
    return out.squeeze()

### 1.3 Define Generator model

In [22]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            nn.Linear(110, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

### 1.4 Define Conditional GAN

In [23]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

### 1.5 Define Training params

In [24]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

writer = SummaryWriter('')

### 1.6 Generator Training procedure

In [25]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion, device='cpu'):

  g_optimizer.zero_grad()
  z = torch.randn(batch_size, 100).to(device)
  fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).to(device)
  fake_images = generator(z, fake_labels)
  validity = discriminator(fake_images, fake_labels)
  g_loss = criterion(validity, torch.ones(batch_size).to(device))
  g_loss.backward()
  g_optimizer.step()
  return g_loss.item()

### 1.7 Discriminator Training procedure

In [26]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels, device='cpu'):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, torch.ones(batch_size).to(device))

    # train with fake images
    z = torch.randn(batch_size, 100).to(device)
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).to(device)
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, torch.zeros(batch_size).to(device))

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.item()

### 1.8 Conditional GAN training loop

In [27]:
num_epochs = 50
n_critic = 5
display_step = 10
for epoch in range(num_epochs):
    print('Epoch {} ...'.format(epoch), end=' ')
    for i, (images, labels) in enumerate(train_loader):

        step = epoch * len(train_loader) + i + 1
        real_images = images.to(device)
        labels = labels.to(device)
        generator.train()

        d_loss = 0
        for _ in range(n_critic):
            d_loss = discriminator_train_step(len(real_images), discriminator,
                                              generator, d_optimizer, criterion,
                                              real_images, labels, device=device)


        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion, device=device)

        writer.add_scalars('scalars', {'g_loss': g_loss, 'd_loss': (d_loss / n_critic)}, step)

        if step % display_step == 0:
            generator.eval()
            z = torch.randn(9, 100).to(device)
            labels = torch.LongTensor(np.arange(9)).to(device)
            sample_images = generator(z, labels).unsqueeze(1)
            grid = make_grid(sample_images, nrow=3, normalize=True)
            writer.add_image('sample_image', grid, step)
    print('Done!')

Epoch 0 ... Done!
Epoch 1 ... Done!
Epoch 2 ... Done!
Epoch 3 ... Done!
Epoch 4 ... Done!
Epoch 5 ... Done!
Epoch 6 ... Done!
Epoch 7 ... Done!
Epoch 8 ... Done!
Epoch 9 ... Done!
Epoch 10 ... Done!
Epoch 11 ... Done!
Epoch 12 ... Done!
Epoch 13 ... Done!
Epoch 14 ... Done!
Epoch 15 ... Done!
Epoch 16 ... Done!
Epoch 17 ... Done!
Epoch 18 ... Done!
Epoch 19 ... Done!
Epoch 20 ... Done!
Epoch 21 ... Done!
Epoch 22 ... Done!
Epoch 23 ... Done!
Epoch 24 ... Done!
Epoch 25 ... Done!
Epoch 26 ... Done!
Epoch 27 ... Done!
Epoch 28 ... Done!
Epoch 29 ... Done!
Epoch 30 ... Done!
Epoch 31 ... Done!
Epoch 32 ... Done!
Epoch 33 ... Done!
Epoch 34 ... Done!
Epoch 35 ... Done!
Epoch 36 ... Done!
Epoch 37 ... Done!
Epoch 38 ... Done!
Epoch 39 ... Done!
Epoch 40 ... Done!
Epoch 41 ... Done!
Epoch 42 ... Done!
Epoch 43 ... Done!
Epoch 44 ... Done!
Epoch 45 ... Done!
Epoch 46 ... Done!
Epoch 47 ... Done!
Epoch 48 ... Done!
Epoch 49 ... Done!


### 1.9 Generate Data

In [3]:
def generate_digit(generator, digit):
    z = torch.randn(1, 100).to(device)
    label = torch.LongTensor([digit]).to(device)
    img = generator(z, label).data.cpu()
    img = 0.5 * img + 0.5
    return transforms.ToPILImage()(img)

In [29]:
generator.eval()
generate_digit(generator, 2)

## 2. Structure of a Bidirectional Generative Adversarial Network (BiGAN)
A BiGAN, or Bidirectional GAN, is a type of generative adversarial network where the generator not only maps latent samples to generated data, but also has an inverse mapping from data to the latent representation.
![](https://ar5iv.labs.arxiv.org/html/1801.04271/assets/bigan.png)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 2.1 Load Data

In [4]:
batch_size = 128
mnist_train, mnist_test = load_dataset(batch_size=batch_size)

### 2.2 Define Generator

- **Role:** Takes a latent vector \( z \) (sampled from a simple prior, like a Gaussian) and generates a synthetic data sample \( G(z) \) (e.g., an image).
- **Goal:** Produce realistic outputs so that the discriminator cannot distinguish them from real data.

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(50, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.layers(z)

### 2.3 Define Discriminator

- **Role:** Receives pairs of data and latent codes. It sees:
  - **Real pair:** \((x, E(x))\) where \( x \) is a real data sample and \( E(x) \) is its encoded latent representation.
  - **Fake pair:** \((G(z), z)\) where \( z \) is a sampled latent vector and \( G(z) \) is the generated data sample.
- **Goal:** Distinguish between real and fake pairs by outputting a probability that a given pair is real.


In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(28 * 28 + 50, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024,1),
            nn.Sigmoid()
        )

    def forward(self, X, z):
        Xz = torch.cat([X, z], dim=1)
        return self.layers(Xz)

### 2.4 Define Encoder

- **Role:** Maps a real data sample \( x \) (e.g., an image) to a latent representation \( E(x) \).
- **Goal:** Learn an inverse mapping of the generator, so that the latent code encapsulates meaningful information about the data.

In [8]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(28 * 28, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 50)
        )

    def forward(self, X):
        return self.layers(X)


### 2.5 Define Loss function for discriminator and Encoder-Generator

4. **Discriminator Training:**
   - **Objective:** Update $( D )$ so that it assigns a high probability (close to 1) to the real pair and a low probability (close to 0) to the fake pair.


5. **Generator and Encoder Training:**
   - **Objective:** Update $( G )$ and $( E )$ together so that the fake pairs $((G(z), z))$ become more similar to the real pairs $((x, E(x)))$, effectively “fooling” $(D)$

In [8]:
def D_loss(DG, DE, eps=1e-6):
    loss = torch.log(DE + eps) + torch.log(1 - DG + eps)
    return -torch.mean(loss)

In [9]:
def EG_loss(DG, DE, eps=1e-6):
    loss = torch.log(DG + eps) + torch.log(1 - DE + eps)
    return -torch.mean(loss)

In [10]:
def init_weights(Layer):
    name = Layer.__class__.__name__
    if name == 'Linear':
        torch.nn.init.normal_(Layer.weight, mean=0, std=0.02)
        if Layer.bias is not None:
            torch.nn.init.constant_(Layer.bias, 0)

### 2.6 Define BiGAN and Training Params

In [12]:
l_rate = 2e-5

E = Encoder().to(device)
G = Generator().to(device)
D = Discriminator().to(device)

E.apply(init_weights)
G.apply(init_weights)
D.apply(init_weights)

#optimizers with weight decay
optimizer_EG = torch.optim.Adam(list(E.parameters()) + list(G.parameters()),
                                lr=l_rate, betas=(0.5, 0.999), weight_decay=1e-5)
optimizer_D = torch.optim.Adam(D.parameters(),
                               lr=l_rate, betas=(0.5, 0.999), weight_decay=1e-5)

## 3. Task 2
```
Task 2.1
- Train defined BiGAN above
- In the training procedure add tensorboard and every 10 epochs visualize 10
  - generated images
  - reconstructed images
```
<hr>

```
Task 2.2
- Implement and train a conditional BiGAN for CIFAR10 dataset
```

## Task 2.1

In [13]:
writer = SummaryWriter('', filename_suffix='_task2.1')

In [14]:
def generator_encoder_train_step(discriminator, generator, encoder, real_images, g_optimizer, device='cpu'):

  g_optimizer.zero_grad()
  z = 2 * torch.rand(real_images.size(0), 50) - 1
  z = z.to(device)
  fake_images = generator(z)
  latent_vector = encoder(real_images)

  DG = discriminator(fake_images, z)
  DE = discriminator(real_images, latent_vector)

  loss_EG = EG_loss(DG, DE)

  loss_EG.backward()
  g_optimizer.step()
  return loss_EG.item()

In [15]:
def discriminator_train_step(discriminator, generator, encoder, d_optimizer, real_images, device='cpu'):
    d_optimizer.zero_grad()

    # train with real images
    latent_vector = encoder(real_images)
    DE = discriminator(real_images, latent_vector)

    # train with fake images
    z = 2 * torch.rand(real_images.size(0), 50) - 1
    z = z.to(device)
    fake_images = generator(z)
    DG = discriminator(fake_images, z)
    
    #compute losses
    loss_D = D_loss(DG, DE)
   
    loss_D.backward()
    d_optimizer.step()
    return loss_D.item()

In [42]:
num_epochs = 30
n_critic = 5
display_step = 1

for epoch in range(1, num_epochs):
    D.train()
    E.train()
    G.train()
    
    for i, (images, _) in enumerate(mnist_train):
        step = (epoch - 1) * len(mnist_train) + i + 1
        real_images = images.reshape(images.size(0),-1).to(device)
        eg_loss = generator_encoder_train_step(D, G, E, real_images, optimizer_EG, device=device)
        loss_D = 0
        for _ in range(n_critic):
            loss_D += discriminator_train_step(D, G, E,
                                              optimizer_D,
                                              real_images, device=device)
        d_loss = loss_D / n_critic

        
    if epoch % display_step == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Avg_Loss_D: {d_loss:.4f}, Avg_Loss_EG: {eg_loss:.4f}')
        writer.add_scalars('scalars', {'Avg_Loss_EG': eg_loss, 'Avg_Loss_D': d_loss}, epoch)

        n_show = 10
        D.eval()
        E.eval()
        G.eval()
        
        with torch.no_grad():
            real = images[:n_show].reshape(n_show,-1).to(device)
            z = 2 * torch.rand(n_show, 50) - 1
            z = z.to(device)
            gener = G(z).reshape(n_show, 28, 28).cpu().numpy()
            recon = G(E(real)).reshape(n_show, 28, 28).cpu().numpy()
            real = real.reshape(n_show, 28, 28).cpu().numpy()

            fig, ax = plt.subplots(3, n_show, figsize=(15,5))
            fig.subplots_adjust(wspace=0.05, hspace=0)
            plt.rcParams.update({'font.size': 20})
            fig.suptitle(f'Epoch {epoch}')
            fig.text(0.04, 0.75, 'G(z)', ha='left')
            fig.text(0.04, 0.5, 'x', ha='left')
            fig.text(0.04, 0.25, 'G(E(x))', ha='left')

            for i in range(n_show):
                ax[0, i].imshow(gener[i], cmap='gray')
                ax[0, i].axis('off')
                ax[1, i].imshow(real[i], cmap='gray')
                ax[1, i].axis('off')
                ax[2, i].imshow(recon[i], cmap='gray')
                ax[2, i].axis('off')
            
            writer.add_figure('sample_image', fig, step)
    print('Done!')

Epoch [1/30], Avg_Loss_D: 0.2564, Avg_Loss_EG: 4.3752
Done!
Epoch [2/30], Avg_Loss_D: 0.0914, Avg_Loss_EG: 6.7360
Done!
Epoch [3/30], Avg_Loss_D: 0.0986, Avg_Loss_EG: 6.5528
Done!
Epoch [4/30], Avg_Loss_D: 0.0930, Avg_Loss_EG: 7.2560
Done!
Epoch [5/30], Avg_Loss_D: 0.0497, Avg_Loss_EG: 9.0924
Done!
Epoch [6/30], Avg_Loss_D: 0.2029, Avg_Loss_EG: 5.9365
Done!
Epoch [7/30], Avg_Loss_D: 0.0995, Avg_Loss_EG: 6.8211
Done!
Epoch [8/30], Avg_Loss_D: 0.0325, Avg_Loss_EG: 9.0642
Done!
Epoch [9/30], Avg_Loss_D: 0.0771, Avg_Loss_EG: 7.9964
Done!
Epoch [10/30], Avg_Loss_D: 0.0440, Avg_Loss_EG: 9.0893
Done!
Epoch [11/30], Avg_Loss_D: 0.0412, Avg_Loss_EG: 8.6907
Done!
Epoch [12/30], Avg_Loss_D: 0.0227, Avg_Loss_EG: 11.5688
Done!
Epoch [13/30], Avg_Loss_D: 0.1460, Avg_Loss_EG: 7.5683
Done!
Epoch [14/30], Avg_Loss_D: 0.0533, Avg_Loss_EG: 8.7658
Done!
Epoch [15/30], Avg_Loss_D: 0.0422, Avg_Loss_EG: 8.9357
Done!
Epoch [16/30], Avg_Loss_D: 0.0543, Avg_Loss_EG: 9.7516
Done!
Epoch [17/30], Avg_Loss_D: 0.091

## Task 2.2

In [11]:
class Generator(nn.Module):
    def __init__(self, latent_dim=50, embedding_dim=10):
        super(Generator, self).__init__()
        self.label_emb = nn.Embedding(10, embedding_dim)
        self.fc = nn.Sequential(
            nn.Linear(latent_dim + embedding_dim, 256 * 4 * 4),
            nn.ReLU(True)
        )
        self.deconv = nn.Sequential(
            # Input: (256, 4, 4) -> output: (128, 8, 8)
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # (128, 8, 8) -> (64, 16, 16)
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # (64, 16, 16) -> (3, 32, 32)
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )
        
    def forward(self, z, labels):
        # z: (batch, latent_dim); labels: (batch,)
        e = self.label_emb(labels)               # (batch, embedding_dim)
        x = torch.cat([z, e], dim=1)             # (batch, latent_dim + embedding_dim)
        x = self.fc(x)                           # (batch, 256*4*4)
        x = x.view(x.size(0), 256, 4, 4)         # (batch, 256, 4, 4)
        x = self.deconv(x)                       # (batch, 3, 32, 32)
        return x


class Discriminator(nn.Module):
    def __init__(self, embedding_dim=10):
        super(Discriminator, self).__init__()
        self.label_emb = nn.Embedding(10, embedding_dim)
        
        self.label_fc = nn.Linear(embedding_dim, 32 * 32)
        
        self.conv = nn.Sequential(
            # Input: (3 channels) + label (1 channel)
            nn.Conv2d(4, 64, kernel_size=4, stride=2, padding=1, bias=False),  # (64, 16, 16)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False), # (128, 8, 8)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False), # (256, 4, 4)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Linear(256 * 4 * 4, 1),
            nn.Sigmoid()
        )
        
    def forward(self, X, z, labels):
        # X: (batch, 3, 32, 32); labels: (batch,)
        batch_size = X.size(0)
        e = self.label_emb(labels)             # (batch, embedding_dim)
        e = self.label_fc(e)                   # (batch, 32*32)
        e = e.view(batch_size, 1, 32, 32)      # (batch, 1, 32, 32)
        x_cond = torch.cat([X, e], dim=1)      # (batch, 3+1=4, 32, 32)
        out = self.conv(x_cond)
        out = out.view(batch_size, -1)
        out = self.fc(out)
        return out


class Encoder(nn.Module):
    def __init__(self, embedding_dim=10):
        super(Encoder, self).__init__()
        self.label_emb = nn.Embedding(10, embedding_dim)
        self.label_fc = nn.Linear(embedding_dim, 32 * 32)
        
        self.conv = nn.Sequential(
            # Input: (3 channels) + label (1 channel)
            nn.Conv2d(4, 64, kernel_size=4, stride=2, padding=1),  # (64, 16, 16)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # (128, 8, 8)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # (256, 4, 4)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.fc = nn.Linear(256 * 4 * 4, 50)
        
    def forward(self, X, labels):
        batch_size = X.size(0)
        e = self.label_emb(labels)              # (batch, embedding_dim)
        e = self.label_fc(e)                    # (batch, 32*32)
        e = e.view(batch_size, 1, 32, 32)       # (batch, 1, 32, 32)
        x_cond = torch.cat([X, e], dim=1)       # (batch, 4, 32, 32)
        out = self.conv(x_cond)
        out = out.view(batch_size, -1)
        latent = self.fc(out)
        return latent

    

def discriminator_train_step(discriminator, generator, encoder, d_optimizer, real_images, real_labels, device='cpu'):
    d_optimizer.zero_grad()

    # train with real images
    latent_vector = encoder(real_images, real_labels)
    DE = discriminator(real_images, latent_vector, real_labels)

    # train with fake images
    z = 2 * torch.rand(images.size(0), 50) - 1
    z = z.to(device)
    fake_labels = torch.LongTensor(np.random.randint(0, 10, real_images.size(0))).to(device)
    fake_images = generator(z, fake_labels)
    DG = discriminator(fake_images, z, fake_labels)
    
    #compute losses
    loss_D = D_loss(DG, DE)
   
    loss_D.backward()
    d_optimizer.step()
    return loss_D.item()


def generator_encoder_train_step(discriminator, generator, encoder, real_images, real_labels, g_optimizer, device='cpu'):

    g_optimizer.zero_grad()
    z = 2 * torch.rand(real_images.size(0), 50) - 1
    z = z.to(device)
    fake_labels = torch.LongTensor(np.random.randint(0, 10, real_images.size(0))).to(device)
    fake_images = generator(z, fake_labels)
    latent_vector = encoder(real_images, real_labels)

    DG = discriminator(fake_images, z, fake_labels)
    DE = discriminator(real_images, latent_vector, real_labels)

    loss_EG = EG_loss(DG, DE)

    loss_EG.backward()
    g_optimizer.step()
    return loss_EG.item()


In [15]:
l_rate = 2e-5

E = Encoder().to(device)
G = Generator().to(device)
D = Discriminator().to(device)

E.apply(init_weights)
G.apply(init_weights)
D.apply(init_weights)

#optimizers with weight decay
optimizer_EG = torch.optim.Adam(list(E.parameters()) + list(G.parameters()),
                                lr=l_rate, betas=(0.5, 0.999), weight_decay=1e-5)
optimizer_D = torch.optim.Adam(D.parameters(),
                               lr=l_rate, betas=(0.5, 0.999), weight_decay=1e-5)

writer = SummaryWriter('', filename_suffix='_task2.2')

In [16]:
batch_size = 128
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
])


def load_dataset(batch_size = 128, root = '~/datasets/', transform=transforms.ToTensor()):
    train_dataset = torchvision.datasets.CIFAR10(root = root + '/CIFAR10', train=True,
                                               transform=transform, download=True)

    test_dataset = torchvision.datasets.CIFAR10(root = root + '/CIFAR10', train=False,
                                              transform=transform)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

cifar_loader, _ = load_dataset(batch_size=batch_size, transform=transform)

In [17]:
num_epochs = 30
n_critic = 5
n_generate = 1
display_step = 1

for epoch in range(1, num_epochs):
    D.train()
    E.train()
    G.train()
    
    for i, (images, labels) in enumerate(cifar_loader):
        step = (epoch - 1) * len(cifar_loader) + i + 1
        real_images = images.to(device)
        labels = labels.to(device)
        
        loss_D = 0
        for _ in range(n_critic):
            loss_D += discriminator_train_step(D, G, E,
                                              optimizer_D,
                                              real_images, labels, device=device)
        d_loss = loss_D / n_critic
            
        eg_loss = 0
        for _ in range(n_generate):
            eg_loss += generator_encoder_train_step(D, G, E, real_images, labels, optimizer_EG, device=device)
        eg_loss = eg_loss / n_generate

        
    if epoch % display_step == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Avg_Loss_D: {d_loss:.4f}, Avg_Loss_EG: {eg_loss:.4f}')
        writer.add_scalars('scalars', {'Avg_Loss_EG': eg_loss, 'Avg_Loss_D': d_loss}, epoch)

        n_show = 10
        D.eval()
        E.eval()
        G.eval()
        
        with torch.no_grad():
            real = images[:n_show].to(device)
            real_labels = labels[:n_show].to(device)
            z = 2 * torch.rand(n_show, 50) - 1
            z = z.to(device)
            fake_labels_vis = torch.randint(0, 10, (n_show,), device=device)
            gener = G(z, fake_labels_vis).reshape(n_show, 3, 32, 32).permute(0 ,2,3,1).cpu().numpy()
            recon = G(E(real, real_labels), real_labels).reshape(n_show, 3, 32, 32).permute(0,2,3,1).cpu().numpy()
            real = real.reshape(n_show, 3, 32, 32).permute(0,2,3,1).cpu().numpy()

            normalize = lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8)
            gener = normalize(gener)
            recon = normalize(recon)
            real = normalize(real)

            fig, ax = plt.subplots(3, n_show, figsize=(15,5))
            fig.subplots_adjust(wspace=0.05, hspace=0)
            plt.rcParams.update({'font.size': 20})
            fig.suptitle(f'Epoch {epoch}')
            fig.text(0.04, 0.75, 'G(z)', ha='left')
            fig.text(0.04, 0.5, 'x', ha='left')
            fig.text(0.04, 0.25, 'G(E(x))', ha='left')

            for i in range(n_show):
                ax[0, i].imshow(gener[i])
                ax[0, i].axis('off')
                ax[1, i].imshow(real[i])
                ax[1, i].axis('off')
                ax[2, i].imshow(recon[i])
                ax[2, i].axis('off')
            
            writer.add_figure('sample_image', fig, step)
    print('Done!')

Epoch [1/30], Avg_Loss_D: 0.0043, Avg_Loss_EG: 14.7523
Done!
Epoch [2/30], Avg_Loss_D: 0.0022, Avg_Loss_EG: 17.7251
Done!
Epoch [3/30], Avg_Loss_D: 0.0021, Avg_Loss_EG: 19.5404
Done!
Epoch [4/30], Avg_Loss_D: 0.0033, Avg_Loss_EG: 14.7663
Done!
Epoch [5/30], Avg_Loss_D: 0.0005, Avg_Loss_EG: 19.7046
Done!
Epoch [6/30], Avg_Loss_D: 0.0057, Avg_Loss_EG: 14.9596
Done!
Epoch [7/30], Avg_Loss_D: 0.0031, Avg_Loss_EG: 18.3187
Done!
Epoch [8/30], Avg_Loss_D: 0.0011, Avg_Loss_EG: 17.8246
Done!
Epoch [9/30], Avg_Loss_D: 0.0031, Avg_Loss_EG: 14.4047
Done!
Epoch [10/30], Avg_Loss_D: 0.0088, Avg_Loss_EG: 13.1464
Done!
Epoch [11/30], Avg_Loss_D: 0.0032, Avg_Loss_EG: 15.7511
Done!
Epoch [12/30], Avg_Loss_D: 0.0057, Avg_Loss_EG: 14.6656
Done!
Epoch [13/30], Avg_Loss_D: 0.0080, Avg_Loss_EG: 14.2493
Done!
Epoch [14/30], Avg_Loss_D: 0.0051, Avg_Loss_EG: 14.6086
Done!
Epoch [15/30], Avg_Loss_D: 0.0092, Avg_Loss_EG: 13.9169
Done!
Epoch [16/30], Avg_Loss_D: 0.0124, Avg_Loss_EG: 14.7024
Done!
Epoch [17/30], Av

## Resources

* [Simple Explaination of BiGAN](https://youtu.be/rzpA0H-q_HY)
* [Adversarial feature learning](https://arxiv.org/pdf/1605.09782v7.pdf) -- original BiGAN paper